# 1 - Installs

In [7]:
!pip install kornia 
!pip install fastai
!pip install opencv-python
!pip install imageio
!pip install nibabel
!pip install ipywidgets

     ---------------------------------------- 3.3/3.3 MB 5.8 MB/s eta 0:00:00


# 2 - Imports

In [10]:
import os
import glob
import cv2
import imageio

import numpy as np 
import pandas as pd 
import nibabel as nib
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from ipywidgets import *
from PIL import Image
from matplotlib.pyplot import figure

from fastai.basics import *
from fastai.vision.all import *
from fastai.data.transforms import *

C:\Users\balazs\Anaconda3\envs\liver\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3 - Data preprocessing

In [19]:
 # Create a meta file for the nifti files
    
file_list = []

for dirname, _, filenames in os.walk('D:\Datasets\liverSegmentation\input'):
    for filename in filenames:
        file_list.append((dirname,filename))
        
df_files = pd.DataFrame(file_list, columns=['dirname', 'filename'])
df_files.sort_values(by=['filename'], ascending=True)

,dirname,filename
0,D:\Datasets\liverSegmentation\input\segmentations,segmentation-0.nii
1,D:\Datasets\liverSegmentation\input\segmentations,segmentation-1.nii
2,D:\Datasets\liverSegmentation\input\segmentations,segmentation-10.nii
3,D:\Datasets\liverSegmentation\input\segmentations,segmentation-100.nii
4,D:\Datasets\liverSegmentation\input\segmentations,segmentation-101.nii
...,...,...
227,D:\Datasets\liverSegmentation\input\volume_pt6,volume-95.nii
228,D:\Datasets\liverSegmentation\input\volume_pt6,volume-96.nii
229,D:\Datasets\liverSegmentation\input\volume_pt6,volume-97.nii
230,D:\Datasets\liverSegmentation\input\volume_pt6,volume-98.nii


In [23]:
# Map CT scan and segmentation

df_files["mask_dirname"] = ""
df_files["mask_filename"] = ""

for i in range(int(len(file_list)/2)):
    ct = f"volume-{i}.nii"
    mask = f"segmentation-{i}.nii"
    
    df_files.loc[df_files['filename'] == ct, 'mask_filename'] =  mask
    df_files.loc[df_files['filename'] == ct, 'mask_dirname'] = 'D:\Datasets\liverSegmentation\input\segmentations'
    
# delete segmentation rows
df_files = df_files[df_files.mask_filename != ""].sort_values(by=['filename']).reset_index(drop=True)

In [24]:
df_files

,dirname,filename,mask_dirname,mask_filename
0,D:\Datasets\liverSegmentation\input\volume_pt1,volume-0.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-0.nii
1,D:\Datasets\liverSegmentation\input\volume_pt1,volume-1.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-1.nii
2,D:\Datasets\liverSegmentation\input\volume_pt1,volume-10.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-10.nii
3,D:\Datasets\liverSegmentation\input\volume_pt6,volume-100.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-100.nii
4,D:\Datasets\liverSegmentation\input\volume_pt8,volume-101.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-101.nii
...,...,...,...,...
126,D:\Datasets\liverSegmentation\input\volume_pt6,volume-95.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-95.nii
127,D:\Datasets\liverSegmentation\input\volume_pt6,volume-96.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-96.nii
128,D:\Datasets\liverSegmentation\input\volume_pt6,volume-97.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-97.nii
129,D:\Datasets\liverSegmentation\input\volume_pt6,volume-98.nii,D:\Datasets\liverSegmentation\input\segmentations,segmentation-98.nii
